In [2]:
%matplotlib inline
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
educacion = pd.read_csv('/home/luupesado/7506_Datos/2018/datos_tp/joins/educacion.csv')
genero_y_edad = pd.read_csv('/home/luupesado/7506_Datos/2018/datos_tp/joins/genero_y_edad.csv')

Limpiamos el set de datos de educación para quedarnos con el mayor nivel educativo de cada postulante, tal cual lo hicimos en el tp 1

In [4]:
educacion['nombre'] = educacion['nombre'].astype('string')

In [5]:
import unicodedata
def elimina_tildes(s):
   return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

In [6]:
educacion['nombre'] = educacion['nombre'].apply(lambda x: elimina_tildes(x.decode('utf-8')))

In [7]:
educacion['nombre-estado'] = educacion['nombre'] + ' - ' + educacion['estado']

In [10]:
educacion['nombre-estado'] = educacion['nombre-estado'].astype('category')

In [11]:
categories_order = ['Otro - Abandonado',
 'Otro - En Curso',                                        
 'Otro - Graduado', 
'Secundario - Abandonado',
 'Secundario - En Curso',
 'Secundario - Graduado',
 'Terciario/Tecnico - Abandonado',
 'Terciario/Tecnico - En Curso',
 'Terciario/Tecnico - Graduado',
 'Universitario - Abandonado',
 'Universitario - En Curso',
 'Universitario - Graduado',
'Posgrado - Abandonado',
 'Posgrado - En Curso',
 'Posgrado - Graduado',
 'Master - Abandonado',
 'Master - En Curso',
 'Master - Graduado',
'Doctorado - Abandonado',
 'Doctorado - En Curso',
 'Doctorado - Graduado']

In [12]:
educacion['nombre-estado'] = educacion['nombre-estado'].cat \
                                        .set_categories(categories_order, ordered = True)

In [13]:
educacion = educacion.sort_values('nombre-estado', ascending= False).drop_duplicates(subset='idpostulante', keep = 'first')

Ahora limpiamos el set de datos de genero y edad, tal cual lo hicims el el tp1

In [14]:
import datetime as dt

In [15]:
def strdate_to_age(string):
    
    date = dt.datetime.strptime(string, "%Y-%m-%d")
    today = dt.datetime(2018, 4, 1)
    edad = today.year - date.year
    if today.month >= date.month and today.day >= date.day:
        edad = edad +1
    return edad


In [16]:
genero_y_edad = genero_y_edad.dropna()

In [17]:
genero_y_edad['edad'] = genero_y_edad['fechanacimiento'].apply(lambda x: strdate_to_age(str(x)))

In [18]:
genero_y_edad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 477265 entries, 0 to 498633
Data columns (total 5 columns):
Unnamed: 0         477265 non-null int64
idpostulante       477265 non-null object
fechanacimiento    477265 non-null object
sexo               477265 non-null object
edad               477265 non-null int64
dtypes: int64(2), object(3)
memory usage: 21.8+ MB


Ahora hago el merge

In [19]:
educacion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366395 entries, 649440 to 684313
Data columns (total 5 columns):
Unnamed: 0       366395 non-null int64
idpostulante     366395 non-null object
nombre           366395 non-null object
estado           366395 non-null object
nombre-estado    366395 non-null category
dtypes: category(1), int64(1), object(3)
memory usage: 14.3+ MB


In [20]:
genero_y_edad.tail()

,Unnamed: 0,idpostulante,fechanacimiento,sexo,edad
498625,297737,PmGx0vJ,1997-02-27,FEM,21
498627,297739,3NPqmBx,1990-01-08,FEM,28
498629,297741,wVkvPrQ,1994-02-24,MASC,24
498630,297742,MVrow99,1999-09-27,FEM,19
498633,297745,X95z68O,1998-10-29,MASC,20


In [34]:
del genero_y_edad['Unnamed: 0']
del educacion['Unnamed: 0']

In [35]:
postulantes = pd.merge(genero_y_edad, educacion, on='idpostulante', how='inner')

In [36]:
postulantes.head()

,idpostulante,fechanacimiento,sexo,edad,nombre,estado,nombre-estado
0,NM5M,1970-12-03,FEM,48,Secundario,Graduado,Secundario - Graduado
1,5awk,1962-12-04,FEM,56,Universitario,Graduado,Universitario - Graduado
2,ZaO5,1978-08-10,FEM,40,Universitario,Abandonado,Universitario - Abandonado
3,ZaO5,1978-08-10,FEM,40,Universitario,Abandonado,Universitario - Abandonado
4,NdJl,1969-05-09,MASC,49,Posgrado,En Curso,Posgrado - En Curso


In [37]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 444083 entries, 0 to 444082
Data columns (total 7 columns):
idpostulante       444083 non-null object
fechanacimiento    444083 non-null object
sexo               444083 non-null object
edad               444083 non-null int64
nombre             444083 non-null object
estado             444083 non-null object
nombre-estado      444083 non-null category
dtypes: category(1), int64(1), object(5)
memory usage: 24.1+ MB


Ahora filtramos los postulantes por edad, que no habíamos olvidade de hacer antes del merge. NOs quedamos con los postulantes entre 16 y 70 años. Según vimos en el primer TP había muy pocos datos de gente de otras edades. Además los menores de 16 no pueden trabajar legalmente y si bien la edad jubilatoria no es 70 perdemos muy pocos datos con esta restricvción según vimos en el TP1.

In [38]:
postulantes = postulantes[postulantes['edad'] >=16]
postulantes = postulantes[postulantes['edad'] <=70]

In [39]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443959 entries, 0 to 444082
Data columns (total 7 columns):
idpostulante       443959 non-null object
fechanacimiento    443959 non-null object
sexo               443959 non-null object
edad               443959 non-null int64
nombre             443959 non-null object
estado             443959 non-null object
nombre-estado      443959 non-null category
dtypes: category(1), int64(1), object(5)
memory usage: 24.1+ MB


In [40]:
postulantes['idpostulante'] = postulantes.drop_duplicates(subset = 'idpostulante')

In [41]:
postulantes = postulantes.dropna()

In [42]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 358781 entries, 0 to 444082
Data columns (total 7 columns):
idpostulante       358781 non-null object
fechanacimiento    358781 non-null object
sexo               358781 non-null object
edad               358781 non-null int64
nombre             358781 non-null object
estado             358781 non-null object
nombre-estado      358781 non-null category
dtypes: category(1), int64(1), object(5)
memory usage: 19.5+ MB


In [43]:
postulantes.to_csv('/home/luupesado/7506_Datos/2018/TPsDatos_1C2018/TP2/Archivos_a_usar/postulantes')